In [2]:
if True:
    import os
    os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.3.0 pyspark-shell'
    execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))
    import os
    import sys
else:
    os.environ['SPARK_HOME'] = '/usr/lib/spark'
    sys.path.insert(0, '/usr/lib/spark/python/lib/py4j-0.9-src.zip')
    sys.path.insert(0, '/usr/lib/spark/python/')
    sys.path.insert(0, '/usr/local/lib64/python2.7/site-packages')
    sys.path.insert(0,'/usr/local/lib/python2.7/site-packages')
    
    from pyspark import SparkContext
    from pyspark.sql import SQLContext, HiveContext
    
    try: sc = SparkContext()
    except: None    
    sqlc = SQLContext(sc)
    spark = sqlc

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
import seaborn as sns

In [6]:
import numpy as np
import pandas as pd
import time
import json

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, MapType
import pyspark.sql.functions as F

In [7]:
from operator import add
from pyspark.sql.types import *

In [8]:
from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel
from datetime import datetime

# Load data

In [10]:
df_i2i = spark.read.parquet("lab_12/cache/stest_bp_i2i/")

In [11]:
df_i2i.show(2)

+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|        bp_rating|     bp_rating_err|n_used_item_ratings|predicted_i2i_rating|has_i2i_rating|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+
|    28|   4711|   3.0|         244.0|        7697.0|     unr_51|    inr_101|3.564750454655524|-0.564750454655524|                  4|   3.372809845588842|             1|
|    40|  13667|   3.0|         129.0|        2423.0|     unr_51|    inr_101|3.778930846940237|-0.778930846940237|                  2|  2.4987220990293846|             1|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+-----------

In [12]:
df_i2i.count()

3159915

In [13]:
df_als = spark.read.parquet("lab_12/cache/stest_als25_has_missed")

In [14]:
df_als.show(2)

+------+-------+--------------------+
|userId|movieId|predicted_als_rating|
+------+-------+--------------------+
|185012|  21743|           3.4707344|
|185012|  21479|            3.708692|
+------+-------+--------------------+
only showing top 2 rows



In [15]:
df_als.count()

3154183

In [16]:
df_u2u = spark.read.parquet("lab_12/cache/stest_bp_u2u_top150/")

In [17]:
df_u2u.show(2)

+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|userId|movieId|rating|nr_user_cat|nr_item_cat|n_used_user_ratings|has_u2u_rating|predicted_u2u_rating|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
|    28|   4711|   3.0|     unr_51|    inr_101|                 14|             1|   3.634586279581508|
|    40|  13667|   3.0|     unr_51|    inr_101|                  1|             1|  3.9114890042293844|
+------+-------+------+-----------+-----------+-------------------+--------------+--------------------+
only showing top 2 rows



In [18]:
df_u2u.count()

3159915

# Merge everything in single pivot

In [19]:
df_i2i.registerTempTable("df_i2i")
df_als.registerTempTable("df_als")
df_u2u.registerTempTable("df_u2u")

In [20]:
df_i2i.columns

['userId',
 'movieId',
 'rating',
 'n_user_ratings',
 'n_item_ratings',
 'nr_user_cat',
 'nr_item_cat',
 'bp_rating',
 'bp_rating_err',
 'n_used_item_ratings',
 'predicted_i2i_rating',
 'has_i2i_rating']

In [21]:
df_als.columns

['userId', 'movieId', 'predicted_als_rating']

In [22]:
df_u2u.columns

['userId',
 'movieId',
 'rating',
 'nr_user_cat',
 'nr_item_cat',
 'n_used_user_ratings',
 'has_u2u_rating',
 'predicted_u2u_rating']

In [23]:
q = spark.sql("""
      select
          i.*,
          coalesce(u.n_used_user_ratings, 0.0) n_used_user_ratings,
          coalesce(u.has_u2u_rating, 0) has_u2u_rating,
          coalesce(u.predicted_u2u_rating, i.bp_rating) as predicted_u2u_rating,
          coalesce(a.predicted_als_rating, i.bp_rating) as predicted_als_rating,
          if(a.predicted_als_rating is not null, 1, 0) as has_als_rating
      from df_i2i i
      left join df_u2u u on i.userId = u.userId and i.movieId = u.movieId
      left join df_als a on i.userId = a.userId and i.movieId = a.movieId
      
""")

In [24]:
q.show(2)

+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+-------------------+--------------+--------------------+--------------------+--------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|        bp_rating|     bp_rating_err|n_used_item_ratings|predicted_i2i_rating|has_i2i_rating|n_used_user_ratings|has_u2u_rating|predicted_u2u_rating|predicted_als_rating|has_als_rating|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+-------------------+--------------+--------------------+--------------------+--------------+
|    28|   4711|   3.0|         244.0|        7697.0|     unr_51|    inr_101|3.564750454655524|-0.564750454655524|                  4|   3.372809845588842|             1|               14.0|             1|   3.6

In [25]:
q.cache()

DataFrame[userId: int, movieId: int, rating: float, n_user_ratings: decimal(21,1), n_item_ratings: decimal(21,1), nr_user_cat: string, nr_item_cat: string, bp_rating: double, bp_rating_err: double, n_used_item_ratings: bigint, predicted_i2i_rating: double, has_i2i_rating: int, n_used_user_ratings: decimal(21,1), has_u2u_rating: int, predicted_u2u_rating: double, predicted_als_rating: double, has_als_rating: int]

In [26]:
q.count()

3159915

In [27]:
q = q.coalesce(8)

In [31]:
#q.write.parquet("lab_12/cache/stest_pivot4ensembling",  mode='overwrite')

In [ ]:
#res = q.toPandas()

# Read than get pandas

In [10]:
q = spark.read.parquet("lab_12/cache/stest_pivot4ensembling")

In [11]:
q.show(2)

+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+-------------------+--------------+--------------------+--------------------+--------------+
|userId|movieId|rating|n_user_ratings|n_item_ratings|nr_user_cat|nr_item_cat|        bp_rating|     bp_rating_err|n_used_item_ratings|predicted_i2i_rating|has_i2i_rating|n_used_user_ratings|has_u2u_rating|predicted_u2u_rating|predicted_als_rating|has_als_rating|
+------+-------+------+--------------+--------------+-----------+-----------+-----------------+------------------+-------------------+--------------------+--------------+-------------------+--------------+--------------------+--------------------+--------------+
|    28|   4711|   3.0|         244.0|        7697.0|     unr_51|    inr_101|3.564750454655524|-0.564750454655524|                  4|   3.372809845588842|             1|               14.0|             1|   3.6

In [12]:
q.count()

3159915

In [13]:
qrdd = q.rdd.coalesce(30).cache()

In [ ]:
def make_part_filter(index):
    def part_filter(split_index, iterator):
        if split_index == index:
            for el in iterator:
                yield el
    return part_filter


In [16]:
tmp_dir = "/data/home/taras.svirsky/lab12/tmp/"
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)

In [17]:
import cPickle as pickle

In [ ]:
for part_id in range(qrdd.getNumPartitions()):
    part_rdd = qrdd.mapPartitionsWithIndex(make_part_filter(part_id), True)
    data_from_part_rdd = part_rdd.collect()
    print "partition id: %s elements: %s" % (part_id, data_from_part_rdd)
    print part_id, len(data_from_part_rdd),
    fpath = os.path.join(tmp_dir,"part%6d.pckl" % (part_id,))
    with open(fpath, 'w') as outh:
        pickle.dump(data_from_part_rdd, outh)
    print 'saved to', fpath    

In [19]:
2+2

4

In [21]:
asd=pd.DataFrame.from_records(data_from_part_rdd)

In [25]:
zz=data_from_part_rdd[0]

In [32]:
columns = q.columns

In [34]:
df_list = []
for f in os.listdir(tmp_dir):
    print f
    with open(os.path.join(tmp_dir, f)) as inh:
        rows = pickle.load(inh)
    sdf = pd.DataFrame.from_records(rows, columns=columns)
    df_list.append(sdf)

part     0.pckl
part     1.pckl
part     2.pckl
part     3.pckl
part     4.pckl
part     5.pckl
part     6.pckl
part     7.pckl


In [35]:
sum([len(x) for x in df_list])

3159915

In [36]:
df = pd.concat(df_list)

In [38]:
df.head()

,userId,movieId,rating,n_user_ratings,n_item_ratings,nr_user_cat,nr_item_cat,bp_rating,bp_rating_err,n_used_item_ratings,predicted_i2i_rating,has_i2i_rating,n_used_user_ratings,has_u2u_rating,predicted_u2u_rating,predicted_als_rating,has_als_rating
0,28,4711,3.0,244.0,7697.0,unr_51,inr_101,3.564750,-0.564750,4,3.372810,1,14.0,1,3.634586,3.351330,1
1,40,13667,3.0,129.0,2423.0,unr_51,inr_101,3.778931,-0.778931,2,2.498722,1,1.0,1,3.911489,3.766928,1
2,41,830,4.0,341.0,5050.0,unr_51,inr_101,3.525195,0.474805,9,4.336339,1,17.0,1,3.880655,3.543198,1
3,67,14668,5.0,28.0,25449.0,unr_06_50,inr_101,4.570491,0.429509,1,5.000000,1,18.0,1,4.438281,4.322522,1
4,94,13069,4.0,68.0,5233.0,unr_51,inr_101,3.903274,0.096726,4,4.123857,1,33.0,1,4.220622,3.933161,1


In [39]:
df.to_csv("/data/home/taras.svirsky/lab12/data/stest_pivot4ensemble.csv", index=False)

In [40]:
df.shape

(3159915, 17)